In [1]:
import numpy as np
import pandas
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.model_selection import cross_val_score, ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
import pickle
import matplotlib.pyplot as plt

## Constants

In [2]:
# file_path = "/nfs/lab/varpred/Final_Results/final_matrices/"
file_path = "./"
# t1d_mat = file_path + "T1D_final_short.bed"
t1d_mat = file_path + "t1d_final_short_after_loci_filtering.bed"
# t2d_mat = file_path + "T2D_final_short.bed"
t2d_mat = file_path + "t2d_final_short_after_loci_filtering.bed"
topmed_mat = file_path + "topmed_final.bed"
proba_threshold_high = 0.8
proba_threshold_low = 0.000005

# Data Ingestion

In [3]:
# Raw data from csv
t1d_data_raw = pandas.read_csv(t1d_mat, sep='\s+')

# Splitting labels and data
t1d_labels_raw = t1d_data_raw["Probability"]
t1d_data_raw = t1d_data_raw.drop(['VarID', 'Probability'], axis=1)

# Filtering out variants with all 0s
print("Originally", len(t1d_data_raw), "variants.", "Filtering out", len(t1d_data_raw) - len(t1d_data_raw[(t1d_data_raw.T != 0).any()]), "variants; left with", len(t1d_data_raw[(t1d_data_raw.T != 0).any()]))

t1d_data = t1d_data_raw[(t1d_data_raw.T != 0).any()]  # Only non-zero variants
t1d_labels = t1d_labels_raw[t1d_labels_raw.index.isin(t1d_data.index)]  # Get matching labels

# Resetting index for splitting
t1d_data = t1d_data.reset_index(drop=True)
t1d_labels = t1d_labels.reset_index(drop=True)

proba_threshold_high = 0.01
proba_threshold_low = 0.000006
print("Positive", len(t1d_labels[t1d_labels > proba_threshold_high]), end='; ')
print("Negative", len(t1d_labels[t1d_labels < proba_threshold_low]), end='; ')
print("Unlabelled", len(t1d_labels[(t1d_labels <= proba_threshold_high) & (t1d_labels >= proba_threshold_low)]), end='; ')

Originally 24139 variants. Filtering out 0 variants; left with 24139
Positive 589; Negative 1287; Unlabelled 22263; 

In [4]:
# Categorical labelling
def label_prob(prob, prob_threshold_high, prob_threshold_low):
    if prob > prob_threshold_high:
        return 1
    elif prob < prob_threshold_low:
        return 0
    else:
        return -1

In [5]:
# Removing imbalance columns
t1d_data = t1d_data.drop(columns=[col for col in t1d_data.columns if 'imbal' in col])

# Logistic Regression

In [16]:
# high_prob_thresholds = [0.01, 0.001, 0.0001]
# low_prob_thresholds = [0.000005, 0.000006, 0.000007, 0.000008, 0.00001]
high_prob_thresholds = [0.01]
low_prob_thresholds = [0.000005, 0.000006, 0.000007]

res_columns = [
    "Positive Threshold",
    "Negative Threshold",
    "Average Accuracy Score"
]
res = pandas.DataFrame(columns=res_columns)

model = LogisticRegression(max_iter=5000)

for prob_thresh_high in high_prob_thresholds:
    for prob_thresh_low in low_prob_thresholds:
        t1d_labels_thresh = t1d_labels.apply(lambda row: label_prob(row, prob_thresh_high, prob_thresh_low))
        known_data_bool = t1d_labels_thresh != -1
        known_data = t1d_data.copy(deep=True)
        known_data.insert(len(known_data.columns), 'label', t1d_labels_thresh)
        known_data.insert(len(known_data.columns), 'known', known_data_bool)
        known_data = known_data[known_data.known == True]
        known_labels = known_data['label']
        known_data = known_data.drop(['label', 'known'], axis=1)
        
        score = np.mean(cross_val_score(model, known_data, y=known_labels, verbose=1, n_jobs=1))
        res.loc[res.shape[0]] = [prob_thresh_high, prob_thresh_low, score]

res.to_csv("./logistic_reg_res.csv")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

### Data Normalizing

In [ ]:
# high_prob_thresholds = [0.01, 0.001, 0.0001]
# low_prob_thresholds = [0.000005, 0.000006, 0.000007, 0.000008, 0.00001]
high_prob_thresholds = [0.01]
low_prob_thresholds = [0.000005, 0.000006, 0.000007]

res_columns = [
    "Positive Threshold",
    "Negative Threshold",
    "Average Accuracy Score",
    "Num Positive Variants",
    "Num Negative Variants",
    "Pos/Total ratio",
    "Neg/Total ratio",
    "Pos/Neg ratio"
]
res = pandas.DataFrame(columns=res_columns)

model = LogisticRegression(max_iter=5000)
clf = make_pipeline(Normalizer(), model)

for prob_thresh_high in high_prob_thresholds:
    for prob_thresh_low in low_prob_thresholds:
        t1d_labels_thresh = t1d_labels.apply(lambda row: label_prob(row, prob_thresh_high, prob_thresh_low))
        known_data_bool = t1d_labels_thresh != -1
        known_data = t1d_data.copy(deep=True)
        known_data.insert(len(known_data.columns), 'label', t1d_labels_thresh)
        known_data.insert(len(known_data.columns), 'known', known_data_bool)
        known_data = known_data[known_data.known == True]
        known_labels = known_data['label']
        known_data = known_data.drop(['label', 'known'], axis=1)
        
        # Data stats
        num_pos = len(known_labels[known_labels == 1])
        num_neg = len(known_labels[known_labels == 0])
        pos_tot_ratio = num_pos / len(known_labels)
        neg_tot_ratio = num_neg / len(known_labels)
        pos_neg_ratio = num_pos / num_neg
        
        score = np.mean(cross_val_score(clf, known_data, y=known_labels, verbose=1, n_jobs=1))
        res.loc[res.shape[0]] = [prob_thresh_high, prob_thresh_low, score, num_pos, num_neg, pos_tot_ratio, neg_tot_ratio, pos_neg_ratio]

res.to_csv("./logistic_reg_res_normalizer.csv")

# Logit Transform with Linear Regression

In [8]:
import numpy as np
from sklearn.linear_model import LinearRegression


class LogitRegression(LinearRegression):
    
    def __init__(self, n_jobs=1):
        super().__init__(n_jobs=n_jobs)

    def fit(self, x, p):
        p = np.asarray(p)
        y = np.log(p / (1 - p))
        return super().fit(x, y)

    def predict(self, x):
        y = super().predict(x)
        return 1 / (np.exp(-y) + 1)

model = LogitRegression()
clf = make_pipeline(MinMaxScaler(), model)

score = cross_val_score(clf, t1d_data, y=t1d_labels, verbose=1)
print("Score:", score)

score = cross_val_score(model, t1d_data, y=t1d_labels, verbose=1)
print("Score", score)

score = cross_val_score(LinearRegression(), t1d_data, y=t1d_labels, verbose=1)
print("Score:", score)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   22.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Score: [-0.00717984 -0.01306137 -0.00457341 -0.00399431 -0.00738977]


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   24.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Score [-0.00717984 -0.01306137 -0.00457341 -0.00399431 -0.00738977]
Score: [-0.00385987 -0.01537859 -0.01279765 -0.01204553 -0.01145612]


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   27.3s finished


# Random Forest Regression

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
clf = make_pipeline(Normalizer(), model)

score = cross_val_score(clf, t1d_data, y=t1d_labels, verbose=1, n_jobs=5)
print("Score:", score)

score = cross_val_score(make_pipeline(MinMaxScaler(), model), t1d_data, y=t1d_labels, verbose=1, n_jobs=5)
print("Score:", score)

score = cross_val_score(model, t1d_data, y=t1d_labels, verbose=1, n_jobs=5)
print("Score:", score)

# All Regression Models Comparison

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer

class LogitRegression(LinearRegression):
    
    def __init__(self, n_jobs=1):
        super().__init__(n_jobs=n_jobs)

    def fit(self, x, p):
        p = np.asarray(p)
        y = np.log(p / (1 - p))
        return super().fit(x, y)

    def predict(self, x):
        y = super().predict(x)
        return 1 / (np.exp(-y) + 1)

model = LogitRegression()
clf = make_pipeline(MinMaxScaler(), model)
clf2 = make_pipeline(Normalizer(), model)
clf3 = make_pipeline(StandardScaler(), model)

print("All features logit regression scores")

score = cross_val_score(clf, t1d_data, y=t1d_labels)
print("MinMaxScaler normalized Score:", np.mean(score))

score = cross_val_score(clf2, t1d_data, y=t1d_labels)
print("Normalizer Score:", np.mean(score))

score = cross_val_score(clf3, t1d_data, y=t1d_labels)
print("Standard Scaler Score:", np.mean(score))

score = cross_val_score(model, t1d_data, y=t1d_labels)
print("Logit Regression Score", np.mean(score))

score = cross_val_score(LinearRegression(), t1d_data, y=t1d_labels)
print("Linear regression Score:", np.mean(score))

print("Feature selection logit regression scores")

score = cross_val_score(clf, t1d_data_selected, y=t1d_labels)
print("MinMaxScaler normalized Score:", np.mean(score))

score = cross_val_score(clf2, t1d_data_selected, y=t1d_labels)
print("Normalizer Score:", np.mean(score))

score = cross_val_score(clf3, t1d_data_selected, y=t1d_labels)
print("Standard Scaler Score:", np.mean(score))

score = cross_val_score(model, t1d_data_selected, y=t1d_labels)
print("Logit Regression Score", np.mean(score))

score = cross_val_score(LinearRegression(), t1d_data_selected, y=t1d_labels)
print("Linear regression Score:", np.mean(score))

print("PCA logit regression scores")

score = cross_val_score(clf, t1d_data_pca, y=t1d_labels)
print("MinMaxScaler normalized Score:", np.mean(score))

score = cross_val_score(clf2, t1d_data_pca, y=t1d_labels)
print("Normalizer Score:", np.mean(score))

score = cross_val_score(clf3, t1d_data_pca, y=t1d_labels)
print("Standard Scaler Score:", np.mean(score))

score = cross_val_score(model, t1d_data_pca, y=t1d_labels)
print("Logit Regression Score", np.mean(score))

score = cross_val_score(LinearRegression(), t1d_data_pca, y=t1d_labels)
print("Linear regression Score:", np.mean(score))

# SVC 

In [8]:
from sklearn.svm import SVC
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Split unknown and known data
def sort_data(data, labels):
    known_data_bool = labels != -1
    all_data = data.copy(deep=True)
    all_data.insert(len(all_data.columns), 'label', labels)
    all_data.insert(len(all_data.columns), 'known', known_data_bool)
    all_known_data = all_data[all_data.known == True]
    all_unknown_data = all_data[all_data.known == False]
    known_data = all_known_data.drop(['label', 'known'], axis=1)
    known_labels = all_known_data['label']
    unknown_data = all_unknown_data.drop(['label', 'known'], axis=1)
    unknown_labels = all_unknown_data['label']
    
    return all_known_data, unknown_data, known_data, known_labels, unknown_data, unknown_labels

def cv(model, data, labels):
    all_known_data, unknown_data, known_data, known_labels, unknown_data, unknown_labels = sort_data(data, labels)
    ss = ShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
    splits = ss.split(known_data)
    
    total_acc_score = 0
    total_prec_score = 0
    total_rec_score = 0
    num = 0
    for train_idx, test_idx in splits:
        train_data = known_data.iloc[train_idx].append(unknown_data)
        test_data = known_data.iloc[test_idx]
        train_labels = known_labels.iloc[train_idx].append(unknown_labels)
        test_labels = known_labels.iloc[test_idx]

        model.fit(train_data, train_labels)

        # Predict on test
        pred_labels = model.predict(test_data)

        # Measure accuracy
        acc = accuracy_score(pred_labels, test_labels)
        prec = precision_score(pred_labels, test_labels)
        rec = recall_score(pred_labels, test_labels)
        total_acc_score += acc
        total_prec_score += prec
        total_rec_score += rec
        num += 1

    print("Avg accuracy, precision, recall:", total_acc_score / num, total_prec_score / num, total_rec_score / num)
    return total_acc_score / num, total_prec_score / num, total_rec_score / num

In [9]:
# Raw data from csv
t1d_data_raw = pandas.read_csv(t1d_mat, sep='\s+')

# Splitting labels and data
t1d_labels_raw = t1d_data_raw["Probability"]
t1d_data_raw = t1d_data_raw.drop(['Locus', 'VarID', 'Probability'], axis=1)

# Filtering out variants with all 0s
print("Originally", len(t1d_data_raw), "variants.", "Filtering out", len(t1d_data_raw) - len(t1d_data_raw[(t1d_data_raw.T != 0).any()]), "variants; left with", len(t1d_data_raw[(t1d_data_raw.T != 0).any()]))

t1d_data = t1d_data_raw[(t1d_data_raw.T != 0).any()]  # Only non-zero variants
t1d_labels = t1d_labels_raw[t1d_labels_raw.index.isin(t1d_data.index)]  # Get matching labels

# Resetting index for splitting
t1d_data = t1d_data.reset_index(drop=True)
t1d_labels = t1d_labels.reset_index(drop=True)

# Removing imbalance columns
t1d_data = t1d_data.drop(columns=[col for col in t1d_data.columns if 'imbal' in col])

# Categorical labelling
def label_prob(prob, prob_threshold_high, prob_threshold_low):
    if prob > prob_threshold_high:
        return 1
    elif prob < prob_threshold_low:
        return 0
    else:
        return -1
    


Originally 12380 variants. Filtering out 0 variants; left with 12380


In [25]:
high_prob_thresh = 0.0001
low_prob_thresh = 0.00001
clf = SelfTrainingClassifier(SVC(C = high_prob_thresh, probability=True))

t1d_labels_thresh = t1d_labels.apply(lambda row: label_prob(row, high_prob_thresh, low_prob_thresh))
acc, prec, rec = cv(clf, t1d_data, t1d_labels_thresh)

Avg accuracy, precision, recall: 0.5265717674970344 1.0 0.5265717674970344


In [24]:
high_prob_thresh = 0.0001
low_prob_thresh = 0.00001
print(len(t1d_labels[t1d_labels > high_prob_thresh]))
print(len(t1d_labels[t1d_labels < low_prob_thresh]))

3542
3199


In [23]:
len(t1d_labels)

12380